# The Viewer Component
The interactive viewer for IFC models has been extended from Thomas Paviots' excellent [JupyterRenderer](https://github.com/tpaviot/pythonocc-core/blob/master/src/Display/WebGl/jupyter_renderer.py) and offeres a number of functionalities:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ifcopenshell
import json
import collections
import math
import uuid

import numpy as np
import pymeshlab as ml
from tqdm import tqdm
from ifcopenshell.util.selector import Selector
from ipywidgets import interact
from compas.geometry import oriented_bounding_box_numpy

from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
from OCC.Core.BRepBndLib import brepbndlib_AddOBB
from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

from utils.JupyterIFCRenderer import JupyterIFCRenderer

In [3]:
m = ifcopenshell.open("../merged_full.ifc")
system_dict_file = "../WestDeckBox.nwd_aggregation.json"
#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Relationship Identification

###  aggrgegation relationships

In [ ]:
# flatten info dictionary
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten(el, new_key_2, sep=sep))
                else:
                    
                    items.append(el)
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append(v)
    return (items)


def get_systems(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        out_dict[sys_name] = flatten(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        

get_systems(system_dict_file)    
    
    

### Topological relationships

In [4]:
# flatten info dictionary to branch level
def flatten_branch(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten_branch(el, new_key_2, sep=sep))
                else:
                    
                    items.append((k, el))
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten_branch(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append((k,v))
    return (items)


# get components of each branch
def get_branches(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        items = flatten_branch(sys)
        print(items)
        d = {}
        for branch, e in items:
            if not branch in d:
                d[branch] = []
            d[branch] += [e]
        out_dict[sys_name] = d
        
        #out_dict[sys_name] = flatten_branch(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        
system = 'West-DeckBox-Pipe.rvm'
branches = get_branches(system_dict_file)[system]    


/tmp/ipykernel_5672/1051200402.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(el, collections.MutableMapping):
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[('/DB-13-INT-8-1-B01', 'STRT 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STRT 2 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 2 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'BEND 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'HACC 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'SPLR 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STRT 3 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STRT 4 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 3 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 4 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 5 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'STIFFENER 6 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'BRCO 1 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', 'BRCO 2 of BRANCH /DB-13-INT-8-1-B01'), ('/DB-13-INT-8-1-B01', '

In [8]:


# # draw links between connected elements
# def visualize_branches(branches, ifc, contiguous=False):
#     vis_dict = {}
#     for k, val in branches.items():
#         vis_elements = []
#         for element in val:
#             if element in pipe_names:
#                 vis_elements.append((element, pipe_type))
#             elif element in fitting_names:
#                 vis_elements.append((element, fitting_type))
#         vis_dict[k] = vis_elements

#     error_count = 0
#     # enumerate through branches
#     count = 0
#     for k, val in tqdm(vis_dict.items()):
#         if count == 50:
#             break
#         branch_size = len(val)
#         for i, element in enumerate(val):
#             #check if element is not the last element
#             if (i+1) < branch_size:
#                 try:
#                     draw_relationship(element[0], element[1], 
#                           val[i+1][0], val[i+1][1], ifc)
#                 except:
#                     error_count +=1
#         count +=1

#     print(error_count)
#     #print(vis_dict)
    
def visualize_branches(branches, ifc, contiguous=True):
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    fitting_names = [f.Name for f in fittings]
    pipe_names = [p.Name for p in pipes]
    print(pipes[0].Name)

    vis_dict = {}
    for k, val in branches.items():
        vis_elements = []
        connect = True
        for element in val:
            if element in pipe_names:
                vis_elements.append((element, pipe_type, connect))
                connect = True
            elif element in fitting_names:
                vis_elements.append((element, fitting_type, connect))
                connect = True
            else:
                connect = False
        vis_dict[k] = vis_elements

    error_count = 0
    # enumerate through branches
    count = 0
    for k, val in tqdm(vis_dict.items()):
        if count == 50:
            break
        branch_size = len(val)
        for i, element in enumerate(val):
            #check if element is not the last element
            if (i+1) < branch_size:
                try:
                    if val[i+1][2] or not contiguous:
                        draw_relationship(element[0], element[1], 
                              val[i+1][0], val[i+1][1], ifc)
                except:
                    error_count +=1
        count +=1

    print(error_count)

    
visualize_branches(branches, m)

TUBE 6 of BRANCH /DO-4320691-WD-CDB-01/B1


  0%|                                                                                                         | 0/3563 [00:00<?, ?it/s]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
d
ELBOW 1 of BRANCH /AI-8391004-WD-MDA-02/B1 [0.029170292410500065, 0.023415625095400294, 0.02917017320119797] [-1.0, 0.0, 0.0] [-1.3630547104212098, 12.3117046356201, 55.3076535044047]
TUBE 2 of BRANCH /AI-8391004-WD-MDA-02/B1 [0.016619585443626725, 0.43498277822892994, 0.01661958544362765] [-1.0, 0.0, 0.0] [-1.81600773334503, 12.3117046356201, 55.3018989562988]
rel [-1.3630547104212098, 12.3117046356201, 55.3076535044047] [-1.4810249551161, 12.3117046356201, 55.3018989562988] ELBOW 1 of BRANCH /AI-8391004-WD-MDA-02/B1 TUBE 2 of BRANCH /AI-8391004-WD-MDA-02/B1
True
d
TUBE 2 of BRANCH /AI-8391004-WD-MDA-02/B1 [0.016619585443626725, 0.43498277822892994, 0.01661958544362765] [-1.0, 0.0, 0.0] [-1.81600773334503, 12.3117046356201, 55.3018989562988]
ELBOW 2 of BRANCH /AI-8391004-WD-MDA-02/B1 [0.02917041134838505, 0.02341596255927573

  0%|                                                                                               | 4/3563 [00:05<1:16:18,  1.29s/it]

TUBE 4 of BRANCH /AI-8391004-WD-MDA-02/B1 [0.9864000712655532, 0.01661958544363654, 0.016619585443621698] [0.0, 0.0, -1.0] [-2.2747156620025653, 12.433228492736802, 54.17819213867185]
rel [-2.2747156620025653, 12.427728157881972, 55.17810062091427] [-2.2747156620025653, 12.433228492736802, 55.0645922099374] ELBOW 3 of BRANCH /AI-8391004-WD-MDA-02/B1 TUBE 4 of BRANCH /AI-8391004-WD-MDA-02/B1
False
False
False
False
False
False
False
False
False
True
d
TUBE 2 of BRANCH /AI-8391757-WD-MDA-01/B1 [0.03500000166240014, 0.006319423176119629, 0.006319423176140605] [0.0, 0.0, -1.0] [-1.3572992086410707, 12.311704635620112, 55.8849983215332]


...

TUBE 4 of BRANCH /CB-8441030-WD-MDA-02/B1 [0.016619586079377205, 0.5608713055287168, 0.016619586113199088] [2.416282432289686e-07, 1.655465199814224e-07, -0.999999999999957] [-2.7182841304014618, 12.433227538906205, 54.297370912674495]
rel [-2.7182843685150146, 12.427725658388269, 54.87175475433577] [-2.7182842417609856, 12.433227462610565, 54.75824221820319] ELBOW 3 of BRANCH /CB-8441030-WD-MDA-02/B1 TUBE 4 of BRANCH /CB-8441030-WD-MDA-02/B1
False
False
True
d
ELBOW 1 of BRANCH /CB-8441031-WD-MDA-02/B1 [0.029172080549848545, 0.029170292410501952, 0.023415625095399406] [0.0, 0.0, -1.0] [-1.007454710993418, 12.3082275390625, 55.10765464881385]
TUBE 2 of BRANCH /CB-8441031-WD-MDA-02/B1 [0.35425154270494996, 0.01661958544362721, 0.01661958544362322] [-1.0, 0.0, 0.0] [-1.37967658042908, 12.3082275390625, 55.1018981933594]
rel [-1.007454710993418, 12.3082275390625, 55.10765464881385] [-1.12542503772413, 12.3082275390625, 55.1018981933594] ELBOW 1 of BRANCH /CB-8441031-WD-MDA-02/B1 TUBE 2 of

  1%|█▎                                                                                            | 49/3563 [02:56<1:50:05,  1.88s/it]

TUBE 7 of BRANCH /CB-8441031-WD-MDA-02/B1 [0.01661958544752008, 0.4137124121189011, 0.016619585447494912] [0.0, 0.0, 1.0] [-2.27471613883972, 12.308228492736799, 53.605499280237495]
rel [-2.2712796760770706, 12.312523282227103, 54.03272279399913] [-2.27471613883972, 12.308228492736799, 53.9192116923564] ELBOW 6 of BRANCH /CB-8441031-WD-MDA-02/B1 TUBE 7 of BRANCH /CB-8441031-WD-MDA-02/B1
False
False
True
d
TUBE 2 of BRANCH /CB-8441102-WD-MDA-01/B1 [0.07347600162029977, 0.006319423176164926, 0.006319423176142395] [0.0, 0.0, -1.0] [-1.0016992092132577, 12.20535469055175, 56.2275238039165]


  1%|█▎                                                                                            | 50/3563 [02:57<3:28:04,  3.55s/it]

FLANGE 1 of BRANCH /CB-8441102-WD-MDA-01/B1 [0.04742055246691144, 0.024100001901398826, 0.047420552466895284] [0.7730103394496779, -0.634393422967084, 0.0] [-1.0016992092132568, 12.205354690551747, 56.152748107534904]
rel [-1.0016992092132577, 12.20535469055175, 56.2275238039165] [-1.0016992092132568, 12.205354690551747, 56.152748107534904] TUBE 2 of BRANCH /CB-8441102-WD-MDA-01/B1 FLANGE 1 of BRANCH /CB-8441102-WD-MDA-01/B1
8


## Visualization

In [ ]:
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer.setAllTransparent()
viewer

### aggregation relationships

In [ ]:
picker = viewer.colorPicker()
picker


In [ ]:
out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
           'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# PAINT A SET OF ELEMENTS IN ONE COLOUR
def systemSelect(system):
    selector = Selector()
    for e in out_dict[system]:
        element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
        viewer.setColor(element, picker.value)
    return system

interact(systemSelect, system=['HVAC', 'electrical'])

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [ ]:
# reset colours
viewer.setDefaultColors()

### topological relationships



## replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [ ]:
element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element_type = "IFCPIPESEGMENT"

selector = Selector()
element = selector.parse(
    m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

shape = element.Representation.Representations[0].Items[0]
element_coords = np.array(shape.Coordinates.CoordList)
#print(element_coords)
bbox = oriented_bounding_box_numpy(element_coords)
print(bbox)

In [6]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]

# get bounding box of ifc element
def get_oriented_bbox(element_name, element_type):
    # get element and bbox
    selector = Selector()
    element = selector.parse(
        m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

    #print(element.Representation.Representations)
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    #print(element_coords)
    bbox = oriented_bounding_box_numpy(element_coords)
    
    # identify box orientation
    l1 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[1][0], bbox[1][1], bbox[1][2]))
    l2 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[3][0], bbox[3][1], bbox[3][2]))
    l3 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[4][0], bbox[4][1], bbox[4][2]))
    half_lengths = [l1/2, l2/2, l3/2]
    
    dominant_axis = half_lengths.index(max(half_lengths))
    if dominant_axis == 0:
        dominant_direction = vector_norm([bbox[0][i] - bbox[1][i] 
                                          for i in range(3)])
    elif dominant_axis == 1:
        dominant_direction = vector_norm([bbox[0][i] - bbox[3][i] 
                                          for i in range(3)])
    else:
        dominant_direction = vector_norm([bbox[0][i] - bbox[4][i] 
                                          for i in range(3)])

    dominance_ratio = max(half_lengths)/sorted(half_lengths)[-2]
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]

    #print(dominance_ratio, dominant_direction)
    print(element_name, half_lengths, dominant_direction, center)

    #print(center)
    return([dominant_direction, max(half_lengths), 
           half_lengths, center])


def CreateBeam(ifcFile, container, name, section, L, position, 
               direction, owner_history, context):
    Z = 0.,0.,1.
    #print('length', L)
    B1 = ifcFile.createIfcBeam(create_guid(),owner_history , name)
    B1.ObjectType ='beam'
    
    #print(type(position[0]))
    B1_Point =ifcFile.createIfcCartesianPoint ( tuple(position) ) 
    #B1_Point =ifcFile.createIfcCartesianPoint ( (0.0,0.0,0.0) ) 
    B1_Axis2Placement = ifcFile.createIfcAxis2Placement3D(B1_Point)
    B1_Axis2Placement.Axis = ifcFile.createIfcDirection(direction)
    B1_Axis2Placement.RefDirection =ifcFile.createIfcDirection(
        np.cross(direction, Z).tolist())

    B1_Placement = ifcFile.createIfcLocalPlacement(
        container.ObjectPlacement,B1_Axis2Placement)
    B1.ObjectPlacement=B1_Placement
    B1_ExtrudePlacement = ifcFile.createIfcAxis2Placement3D(ifcFile.createIfcCartesianPoint ( (0.,0.,0.) )   )
   
    B1_Extruded=ifcFile.createIfcExtrudedAreaSolid()
    B1_Extruded.SweptArea=section
    B1_Extruded.Position=B1_ExtrudePlacement
    B1_Extruded.ExtrudedDirection = ifcFile.createIfcDirection(Z)
    B1_Extruded.Depth = L

    B1_Repr=ifcFile.createIfcShapeRepresentation()
    B1_Repr.ContextOfItems=context
    B1_Repr.RepresentationIdentifier = 'Body'
    B1_Repr.RepresentationType = 'SweptSolid'
    B1_Repr.Items = [B1_Extruded]
    
    B1_DefShape=ifcFile.createIfcProductDefinitionShape()
    B1_DefShape.Representations=[B1_Repr]
    B1.Representation=B1_DefShape
    
    Flr1_Container = ifcFile.createIfcRelContainedInSpatialStructure(create_guid(),owner_history)
    Flr1_Container.RelatedElements=[B1]
    Flr1_Container.RelatingStructure= container
    
    
def Circle_Section(r, ifcfile):
    B1_Axis2Placement2D =ifcfile.createIfcAxis2Placement2D( 
                          ifcfile.createIfcCartesianPoint( (0.,0.,0.) ) )
    B1_AreaProfile = ifcfile.createIfcCircleProfileDef("AREA")
    B1_AreaProfile.Position = B1_Axis2Placement2D 
    B1_AreaProfile.Radius = r
    return B1_AreaProfile


#get_oriented_bbox(element_name, element_type)

In [7]:

def get_point_along_axis(init_point, axis, half_length, edge_distance):
    #print(half_length)
    return (init_point + axis*(half_length - edge_distance))


def sq_distance(x1, y1, z1, x2, y2, z2):
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


def vector_norm(vec):
    den = math.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    return [vec[0]/den, vec[1]/den, vec[2]/den]


# identify a candidate for an edge of the relationship visualization
def get_corner(obb, center_other, edge_distance):
    direction = obb[0]
    center = obb[3]

    candidate1 = [get_point_along_axis(center[i], direction[i], 
                                    obb[1], edge_distance) 
               for i in range(3)]
    candidate2 = [get_point_along_axis(center[i], direction[i], 
                                    -1*obb[1], -1*edge_distance) 
               for i in range(3)]
    # print(corner1, corner2)
    
    dist1 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate1[0],
                   candidate1[1], candidate1[2])    
    dist2 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate2[0],
                   candidate2[1], candidate2[2]) 
    
    if dist1 < dist2:
        return candidate1
    else:
        return candidate2


def draw_sphere(point, radius, colour, viewer):
    point = gp_Pnt(point)
    ball = BRepPrimAPI_MakeSphere(point, radius).Shape()
    viewer.DisplayShape(ball, shape_color = colour, 
                        transparency=True, opacity=0.8)


def draw_cylinder(p1, p2, radius, colour,  element_name1, element_name2, ifc):
    sectionC1 = Circle_Section(r=radius, ifcfile=ifc)
    name='rel '+element_name1 + ' x ' + element_name2
    ConnectingBeam_1 = CreateBeam(ifc, container=floor, name=name, 
                                  section= sectionC1, 
                                  L=math.sqrt(sq_distance(p1[0],p1[1],p1[2],p2[0],p2[1],p2[2])),
                                  position=([p2[0].item(), p2[1].item(), p2[2].item()]),
                                  direction=((p1[0] - p2[0]).item(), (p1[1] - p2[1]).item(), 
                                             (p1[2] - p2[2]).item()),
                                  owner_history=owner_history, context=context)
#     ifc.create_entity('IfcPipeSegment', GlobalId=ifcopenshell.guid.new(), 
#                       Name='rel '+element_name1 + ' x ' + element_name2)
    
# def draw_cylinder(p1, p2, radius, colour, viewer):
#     x = (p1.X() + p2.X())/2
#     y = (p1.Y() + p2.Y())/2
#     z = (p1.Z() + p2.Z())/2
#     p = gp_Pnt(p2.X(), p2.Y(), p2.Z())
#     v = gp_Dir((p1.X() - p2.X()), (p1.Y() - p2.Y()), (p1.Z() - p2.Z()))
#     ax = gp_Ax2(p, v)
#     h = math.sqrt(sq_distance(p1.X(), p1.Y(), p2.Z(), 
#                               p2.X(), p2.Y(), p2.Z()))
#     print(x,y,z,p,v,ax,h)
    
#     cylinder = BRepPrimAPI_MakeCylinder(ax, radius, h).Shape()
#     viewer.DisplayShape(cylinder, shape_color = colour, 
#                         transparency=True, opacity=0.8)
    

# draw a visual indication of a topological relationship    
def draw_relationship (element_name1, element_type1, 
                       element_name2, element_type2, ifc):
    # define params
    #TODO: dynamically scale sphere_radius
    sphere_radius = 0.03
    colour = 'blue'
    threshold = 0.1
    edge_distance = 0.1
    
    # get bboxes
    obb1 = get_oriented_bbox(element_name1, element_type1)
    obb2 = get_oriented_bbox(element_name2, element_type2)
    
    # if bboxes are roughly square, then use centerpoint, 
    # otherwise use a point closer to the edge
    # NOTE: algorithm might fail if dominance ratio is high but
    # the dominent length is still lower than edge_distance
    #print(obb2[2], obb1[2])
    #print('dist', obb1[1])
    if obb1[1] < threshold:
        corner1 = obb1[3]
    else:
        corner1 = get_corner(obb1, obb2[3], edge_distance)
    if  obb2[1] < threshold:
        corner2 = obb2[3]
    else:
        corner2 = get_corner(obb2, obb1[3], edge_distance)
    #print(corner1.X(),corner1.Y(),corner1.Z(),corner2.X(),corner2.Y(),corner1.Z())
    print('rel', corner1, corner2, element_name1, element_name2)
    
    draw_cylinder(corner1, corner2, sphere_radius, 'green', 
                  element_name1, element_name2, ifc)
#     draw_sphere(corner1, sphere_radius, colour, viewer)
#     draw_sphere(corner2, sphere_radius, 'red', viewer)
 

In [ ]:
element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element1_type = "IFCPIPESEGMENT"
element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element2_type = "IFCPIPEFITTING"

element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element3_type = "IFCPIPESEGMENT"
element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element4_type = "IFCPIPEFITTING"
element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
element5_type = "IFCPIPESEGMENT"

draw_relationship(element1_name, element1_type, 
                  element2_name, element2_type, m)
draw_relationship(element2_name, element2_type, 
                  element3_name, element3_type, m)
draw_relationship(element3_name, element3_type, 
                  element4_name, element4_type, m)
draw_relationship(element4_name, element4_type, 
                  element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

In [9]:
m.write('../edited.ifc')

# Misc

In [ ]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
pdct_shape = ifcopenshell.geom.create_shape(settings, inst=element)
viewer.DisplayShape(ball, shape_color = 'blue', transparency=False, opacity=0.5)

In [ ]:
o_bbox1 = get_oriented_boundingbox(list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element1)])
bbox_center1 = [o_bbox[0].X(), o_bbox[0].Y(), o_bbox[0].Z()]


In [ ]:
viewer.shapedict.values()

In [ ]:
list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]

In [ ]:
viewer._shapes

In [ ]:
BoundingBox([[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]]).xmin

In [ ]:
[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]

In [ ]:
print(len(element.Representation.Representations[0].Items[0].CoordIndex))

In [ ]:
element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## Selecting instances 

The instance object currently selected in the 3D view can be assigned to a avariable using the `.getSelectedProduct()` method. 

Provding the plain varialbe name call sthe `Display()`-method of the instance and prints the line of the original ifc-Datei the  [SPFF-format](https://en.wikipedia.org/wiki/ISO_10303-21).


In [ ]:
selection = viewer.getSelectedProduct()
selection

## Show / Hide Objects
Programatically or interactively selected projects can be hidden or shown in the viewer using `setVisible()`.


In [ ]:
viewer.setVisible(selection, False)
viewer

Opening elements are displayed by default. Let's hide them.

In [ ]:
for opening in m.by_type("IfcOpeningElement"):
    viewer.setVisible(opening, False)

## Coloring Objects
Objects can be colored either by hexadicimal numbers provided as strings as they are common in e.g. in HTML and CSS and can be retrieved from many ressources

In [ ]:

viewer.setColorSelected("#ddffaa")
viewer

If you do not happen to have a color in mind  just call a color picker from the viewer:

In [ ]:
picker = viewer.colorPicker()
picker

In [ ]:
viewer.setColorSelected(picker.value)

## IFC to cloud

In [14]:
ifc = ifcopenshell.open("../deckboxtee.ifc")
element_type = 'IFCPIPEFITTING'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(len(tees))


891


In [13]:
for element in tees:
    # create pymeshlab mesh
    #print(element)
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    print(len(element_coords))

768
768
768
2628
2628
1476
2628
2628
768
768
768
768
768
2628
1476
1476
1476
768
2628
2628
768
768
768
768
768
768
2304
768
768
2304
1476
2628
1476
3309
1476
1476
2628
768
2628
1476
768
768
768
768
2304
768
768
768
1476
2628
2628
2628
1476
2628
2628
768
768
768
768
2304
768
768
768
2628
696
768
1476
2628
768
2628
768
768
768
768
768
2304
768
768
768
768
3309
2628
2628
768
2628
2628
768
768
768
768
768
768
768
768
768
2628
2628
768
2628
768
2628
2628
1476
768
768
768
768
2304
768
768
768
768
768
768
1476
1476
1476
2808
1476
2628
768
768
768
768
2304
768
768
768
2304
768
1476
1476
2628
1476
1476
2628
2628
768
768
768
3300
2304
2304
2304
768
768
768
768
768
768
768
2628
2628
2628
2628
2628
1476
2628
768
768
768
2304
768
768
768
768
1476
768
768
2304
2628
768
768
1476
2628
768
768
768
768
768
768
768
768
768
2628
768
768
2628
2628
1476
768
768
768
768
768
768
768
2628
768
768
2628
1476
1476
1476
1476
2304
768
768
768
768
768
2628
768
768
768
768
768
768
1476
2628
1476
768
768
768
768
2628


### Downsampling



In [ ]:
ifc = ifcopenshell.open("../deckboxtee.ifc")

element_type = 'IFCPIPEFITTING'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(tees[0])

In [ ]:
for element in tqdm(tees):
    # create pymeshlab mesh
    #print(element)
    shape = element.Representation.Representations[0].Items[0]
    element_faces = [np.array([i[0]-1, i[1]-1, i[2]-1]) for i in shape.CoordIndex]
    element_coords = np.array(shape.Coordinates.CoordList)
    print(len(element_faces))
#     mesh = ml.Mesh(element_coords, element_faces)
#     ms = ml.MeshSet()
#     ms.add_mesh(mesh, "x")
#     #ms.save_current_mesh("../output.ply")
    
#     # downsample and reassign
#     ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(2))
#     m = ms.current_mesh()
#     shape.CoordIndex = m.face_matrix().tolist()
#     shape.Coordinates.CoordList = m.vertex_matrix().tolist()
    

In [ ]:
ifc.write('../added.ifc') 

In [ ]:
viewer = JupyterIFCRenderer(ifc, size=(400,300))
viewer

In [ ]:
## meshlab downsampling -DOESNT WORK

In [ ]:

print('input mesh has', mesh.vertex_number(), 'vertex and', mesh.face_number(), 'faces')


#Estimate number of faces to have 100+10000 vertex using Euler

#Simplify the mesh. Only first simplification will be agressive
#ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=numFaces, preservenormal=False)
ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(5))
print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")


m = ms.current_mesh()
print('output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
ms.save_current_mesh('../output.ply')

In [ ]:
print(m.face_matrix())

In [ ]:
shape.CoordIndex = m.face_matrix().tolist()
shape.Coordinates.CoordList = m.vertex_matrix().tolist()